<a href="https://colab.research.google.com/github/LauraSchellekens/TM10007_-private_-group/blob/master/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment template

In [147]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
!pip install sklearn numpy matplotlib


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [0]:
# General packages not sure anymore what I'm actually using lol
import numpy as np 
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import impute
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

In [149]:
# Data loading functions. Uncomment the one you want to use
from adni.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


X = data.drop('label', axis=1) # All data without column 'label'
Y = data['label'] 
Y.replace(('AD', 'CN'), (1, 0), inplace=True) # convert AD and CN to 1 and 0


The number of samples: 855
The number of columns: 268


In [150]:
data

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,label,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_002_S_0413_bl_0,5403.229408,4.679203,2.392463,2.040475,1.375697,1.487644,-0.062859,429,0.364471,2.103334,-1.481394,0.455533,1,6395.491754,9940.032401,14622.897328,4.072445,5.204894,4.841280,3.774545,-0.108664,2.224809,0.498333,0.746542,0.785263,-0.007726,-0.069990,-0.029403,-0.000075,-0.068204,0.008103,-0.632451,-1.092715,-1.402496,1304,210,608,0.153782,0.627844,0.457846,...,0.020347,0.022628,0.000556,0.018271,2.374306e-14,4.104987,7.559967,6.673826e-09,1.741492e-09,1.299429e-09,1.734096e-10,514,1.519472e-09,6.500416e-09,2.307441,1.554520e-09,2.374306e-14,4.104987,7.559967,6.673826e-09,1.741492e-09,1.299429e-09,1.734096e-10,514,1.519472e-09,6.500416e-09,2.307441,1.554520e-09,2.446356e-17,3.969563,4.283295,1.930203e-09,4.762307e-10,3.147900e-10,1.195016e-10,8,4.503026e-10,1.810702e-09,2.003542,4.207831e-10
0_002_S_0559_bl_0,922.459760,4.772376,1.709731,1.437500,0.863074,0.940709,-0.247543,412,0.347258,1.685042,-1.222797,0.377850,1,8432.620063,5549.455518,12050.086848,4.076190,5.228020,4.830720,3.252139,-0.730229,0.348783,0.483188,0.412527,0.529344,-0.001275,-0.127039,-0.054140,0.000178,-0.145082,-0.044920,-0.578344,-0.633490,-0.828795,906,229,317,0.189966,0.455902,0.447878,...,0.010227,0.039959,0.001927,0.009183,6.260213e-15,4.471662,3.674709,2.910256e-09,8.867658e-10,7.190769e-10,4.342644e-11,421,8.276855e-10,2.866830e-09,1.605653,7.019460e-10,6.260213e-15,4.471662,3.674709,2.910256e-09,8.867658e-10,7.190769e-10,4.342644e-11,421,8.276855e-10,2.866830e-09,1.605653,7.019460e-10,2.058198e-17,4.401233,2.048344,1.189167e-09,3.678934e-10,3.325169e-10,6.933179e-13,11,4.3

In [154]:
# split into train (70%) and test (30%) set

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(598, 267) (598,)
(257, 267) (257,)


In [155]:
# Preprocessing: drop feature if too many missing values

X_train = X_train.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_train.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_train)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_train.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_train.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_train.shape, y_train.shape)

105
(598, 261) (598,)


In [156]:
X_train

,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_082_S_4208_bl_0,3669.892466,4.569316,2.254397,1.857464,1.274152,1.302636,0.368896,258,0.335274,1.488568,-0.947476,0.322324,10265.922712,4464.559569,7121.790638,3.041492,4.657959,4.234995,10.741482,0.399634,1.813070,0.510810,0.609092,0.604219,-0.007234,-0.001885,-0.015684,0.001750,0.014315,0.001754,-0.748633,-0.681065,-0.965292,1446,433,955,0.046001,0.386082,0.240119,1.259443,...,0.125215,0.005517,0.000017,0.123100,8.243579e-15,4.057287,4.450890,4.919963e-09,1.194130e-09,8.048484e-10,5.478683e-12,381,1.246246e-09,4.914484e-09,1.894150,1.192299e-09,8.243579e-15,4.057287,4.450890,4.919963e-09,1.194130e-09,8.048484e-10,5.478683e-12,381,1.246246e-09,4.914484e-09,1.894150,1.192299e-09,1.193254e-18,3.875000e+00,-0.978988,4.356051e-10,2.321101e-10,2.337837e-10,7.922455e-12,2,1.857998e-10,4.276827e-10,-0.166485,1.315776e-10
0_036_S_4389_bl_0,2041.510152,4.457861,4.285536,1.567959,1.095561,1.127745,0.195329,372,0.287655,1.372630,-1.503604,0.293946,13789.568503,6543.175230,13220.698608,3.256124,4.739575,4.394410,8.029916,0.364226,2.525233,0.467198,0.516993,0.625851,0.006567,-0.010339,0.029970,0.001626,0.016880,0.006170,-0.601523,-0.710202,-0.994713,1819,451,741,0.093494,0.443679,0.335965,1.068720,...,0.031857,0.015691,0.000193,0.029383,3.644253e-15,3.966080,5.234003,2.796233e-09,6.183161e-10,3.744481e-10,4.322499e-67,1099,8.040605e-10,2.796233e-09,2.000718,7.269944e-10,3.644253e-15,3.966080,5.234003,2.796233e-09,6.183161e-10,3.744481e-10,4.322499e-67,1099,8.040605e-10,2.796233e-09,2.000718,7.269944e-10,1.470626e-17,2.990870e+00,7.613342,1.418731e-09,1.980242e-10,

In [157]:
# Preprocessing:  impute missing values (fill missing)

imputer = impute.SimpleImputer(strategy='median')     # imputer with median, to exclude outliers 
X_train_imp = imputer.fit_transform(X_train)          # impute  
X_train_imp = pd.DataFrame(data=X_train_imp, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df

type(X_train_imp)


pandas.core.frame.DataFrame

In [158]:
# Preprocessing: scaling either standard or robust (removes median and scales data according to quantile range) 

# scaler = preprocessing.StandardScaler()
# scaler.fit_transform(X_train_imp)
# X_train_scaled = scaler.transform(X_train_imp)
robust = preprocessing.RobustScaler()
robust.fit_transform(X_train_imp)
X_train_scaled = robust.transform(X_train_imp)

X_train_scaled = pd.DataFrame(data=X_train_scaled, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df
X_train_scaled



,hf_energy,hf_entropy,hf_kurtosis,hf_max,hf_mean,hf_median,hf_min,hf_peak,hf_quartile_range,hf_range,hf_skewness,hf_std,logf_energy_sigma1,logf_energy_sigma10,logf_energy_sigma5,logf_entropy_sigma1,logf_entropy_sigma10,logf_entropy_sigma5,logf_kurtosis_sigma1,logf_kurtosis_sigma10,logf_kurtosis_sigma5,logf_max_sigma1,logf_max_sigma10,logf_max_sigma5,logf_mean_sigma1,logf_mean_sigma10,logf_mean_sigma5,logf_median_sigma1,logf_median_sigma10,logf_median_sigma5,logf_min_sigma1,logf_min_sigma10,logf_min_sigma5,logf_peak_sigma1,logf_peak_sigma10,logf_peak_sigma5,logf_quartile_range_sigma1,logf_quartile_range_sigma10,logf_quartile_range_sigma5,logf_range_sigma1,...,tf_NGTDM_Coarseness,tf_NGTDM_Complexity,tf_NGTDM_Contrast,tf_NGTDM_Strength,"vf_Frangi_edge_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_edge_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_full_std_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_energy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_entropy_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_kurtosis_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_max_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_mean_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_median_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_min_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_peak_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_range_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_skewness_SR(1.0, 10.0)_SS2.0","vf_Frangi_inner_std_SR(1.0, 10.0)_SS2.0"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_082_S_4208_bl_0,0.556590,-0.196470,0.073847,0.148796,0.372893,0.273530,0.528488,-0.403738,-0.130476,-0.414808,0.134972,-0.301244,0.132409,0.005299,-0.005354,-1.053186,-1.186115,-1.075002,1.044572,0.401008,0.573336,-0.332634,0.105819,-0.368520,-0.774888,0.170058,-0.174208,-0.007244,0.078799,-0.135501,-0.791844,0.279157,-0.251923,0.625620,0.900817,0.835720,-0.943801,-0.568505,-1.169864,0.393214,...,1.088832,-0.367691,-0.144912,1.859720,-0.309914,-0.926122,0.359733,-0.154306,-0.309705,-0.364910,-0.187569,-0.183503,-0.203205,-0.135560,0.287408,-0.174317,-0.309914,-0.926122,0.359733,-0.154306,-0.309705,-0.364910,-0.187569,-0.183503,-0.203205,-0.135560,0.287408,-0.174317,-0.327975,-0.043895,-0.396923,-0.782561,-0.483197,-0.230996,-0.264759,-0.315789,-0.597615,-0.740112,-1.384539,-0.799210
0_036_S_4389_bl_0,-0.078060,-0.416993,0.780100,-0.597190,-0.113123,-0.242519,0.250156,0.448598,-0.314043,-0.634138,-0.871624,-0.439818,0.688334,0.556838,1.129146,-0.770285,-0.961929,-0.797901,0.527602,0.372301,0.935899,-0.879274,-0.258977,-0.163012,0.833558,0.038862,0.763054,-0.054771,0.107545,0.110670,0.195559,0.128468,-0.356591,1.242149,0.984831,0.475147,-0.533172,-0.157416,-0.619456,-0.545680,...,0.059063,-0.187569,-0.102849,0.150423,-0.573024,-1.169357,0.648202,-0.749618,-0.957628,-1.086003,-0.239503,2.478221,-0.752221,-0.738130,0.488162,-0.701161,-0.573024,-1.169357,0.648202,-0.749618,-0.957628,-1.086003,-0.239503,2.478221,-0.752221,-0.738130,0.488162,-0.701161,-0.056136,-0.614217,1.707801,0.377303,-0.587300,-0.808852,-0.325012,1.947368,-0.329610,0.469395,1.458783,0.28